In [2]:
import pandas as pd
import pandasql as ps
import plotly.graph_objects as go
import numpy as np

In [3]:
df = pd.read_csv("results.csv")
df

,rep,algo,size,R,block,section,dur
0,0,cpu,1000,3,NaN,NaN,0.000012
1,0,gpu-v1,1000,3,32.0,total,0.000386
2,0,gpu-v1,1000,3,32.0,kernel,0.000007
3,0,gpu-v2,1000,3,32.0,total,0.000309
4,0,gpu-v2,1000,3,32.0,kernel,0.000008
...,...,...,...,...,...,...,...
4051,7,gpu-v1,64000000,3000,1024.0,kernel,0.210248
4052,7,gpu-v2,64000000,3000,1024.0,total,0.325118
4053,7,gpu-v2,64000000,3000,1024.0,kernel,0.233189
4054,7,gpu-v3,64000000,3000,1024.0,total,0.212129


In [4]:
total_df = ps.sqldf("""
select algo, size, R, block, avg(dur) mean, sqrt(avg(dur*dur)-avg(dur)*avg(dur)) std
from df
where section is null or section == 'total'
group by algo, size, R, block;
""")
total_df

,algo,size,R,block,mean,std
0,cpu,1000,3,NaN,0.000007,0.000002
1,cpu,1000,30,NaN,0.000072,0.000016
2,cpu,1000,300,NaN,0.000704,0.000134
3,gpu-v1,1000,3,32.0,0.004079,0.003789
4,gpu-v1,1000,3,64.0,0.000329,0.000072
...,...,...,...,...,...,...
250,gpu-v3,64000000,3000,128.0,0.218819,0.020917
251,gpu-v3,64000000,3000,192.0,0.208948,0.001857
252,gpu-v3,64000000,3000,256.0,0.214994,0.022023
253,gpu-v3,64000000,3000,512.0,0.209589,0.002884


In [5]:
hypers_df = ps.sqldf("""
select size, R, algo, mean, std
from total_df
group by algo, size, R
having mean = min(mean)
order by size, R;
""")
hypers_df

,size,R,algo,mean,std
0,1000,3,cpu,0.000007,0.000002
1,1000,3,gpu-v1,0.000322,0.000058
2,1000,3,gpu-v2,0.000324,0.000062
3,1000,3,gpu-v3,0.000322,0.000058
4,1000,30,cpu,0.000072,0.000016
5,1000,30,gpu-v1,0.000333,0.000046
6,1000,30,gpu-v2,0.000331,0.000046
7,1000,30,gpu-v3,0.000327,0.000042
8,1000,300,cpu,0.000704,0.000134
9,1000,300,gpu-v1,0.000365,0.000068


In [6]:
best_df = ps.sqldf("""
select size, R, algo, mean, std
from hypers_df
group by size, R
having mean = min(mean);
""")
best_df

,size,R,algo,mean,std
0,1000,3,cpu,0.000007,0.000002
1,1000,30,cpu,0.000072,0.000016
2,1000,300,gpu-v3,0.000337,0.000038
3,1000,3000,gpu-v3,0.000367,0.000047
4,1000000,3,gpu-v3,0.001617,0.000095
5,1000000,30,gpu-v1,0.001632,0.000103
6,1000000,300,gpu-v3,0.001941,0.000223
7,1000000,3000,gpu-v3,0.004228,0.000931
8,64000000,3,gpu-v3,0.087934,0.001498
9,64000000,30,gpu-v1,0.089321,0.003173


In [8]:
speedup_df = ps.sqldf("""
select H.size, H.R, H.algo, H.mean, H.mean/B.mean "speedup"
from hypers_df H
  join best_df B
    on H.size = B.size
    and H.R = B.R
where H.mean <> B.mean;
""")
speedup_df

,size,R,algo,mean,speedup
0,1000,3,gpu-v1,0.000322,47.839789
1,1000,3,gpu-v2,0.000324,48.112653
2,1000,3,gpu-v3,0.000322,47.823144
3,1000,30,gpu-v1,0.000333,4.606753
4,1000,30,gpu-v2,0.000331,4.579924
5,1000,30,gpu-v3,0.000327,4.517083
6,1000,300,cpu,0.000704,2.088723
7,1000,300,gpu-v1,0.000365,1.080781
8,1000,300,gpu-v2,0.000362,1.073120
9,1000,3000,gpu-v1,0.000407,1.109195


In [9]:
xy_values = ps.sqldf("""
select distinct size, R
from hypers_df;
""")
xy_values

,size,R
0,1000,3
1,1000,30
2,1000,300
3,1000,3000
4,1000000,3
5,1000000,30
6,1000000,300
7,1000000,3000
8,64000000,3
9,64000000,30


In [11]:
data = []

max_mean = hypers_df["mean"].max()

for algo in hypers_df["algo"].unique():
    algo_df = ps.sqldf(f"""
    select * 
    from hypers_df
    where algo = '{algo}';
    """)

    filled_df = ps.sqldf(f"""
    select XY.size size, XY.R R, ifnull(mean, {max_mean}) mean
    from xy_values XY natural left join algo_df A;
    """)

    filled_df = filled_df.pivot(index="size", columns="R", values="mean")
    print(algo, filled_df)

    surf = go.Surface(y=filled_df.index, x=filled_df.columns, z=filled_df, name=algo)
    data.append(surf)

fig = go.Figure(data=data)
fig.update_layout(title="Duration for all setups", scene=dict(xaxis_title="R", xaxis_type="log", yaxis_title="Size", yaxis_type="log", zaxis_title="Duration", zaxis_type="log"))
fig.write_html("plots.html")

cpu R             3         30        300       3000
size                                            
1000      0.000007  0.000072  0.000704  0.300645
1000000   0.300645  0.300645  0.300645  0.300645
64000000  0.300645  0.300645  0.300645  0.300645
gpu-v1 R             3         30        300       3000
size                                            
1000      0.000322  0.000333  0.000365  0.000407
1000000   0.001826  0.001632  0.002076  0.005285
64000000  0.088614  0.089321  0.106801  0.297975
gpu-v2 R             3         30        300       3000
size                                            
1000      0.000324  0.000331  0.000362  0.000396
1000000   0.002430  0.001885  0.002109  0.005955
64000000  0.093086  0.094596  0.114755  0.300645
gpu-v3 R             3         30        300       3000
size                                            
1000      0.000322  0.000327  0.000337  0.000367
1000000   0.001617  0.002131  0.001941  0.004228
64000000  0.087934  0.090745  0.101088  0.20